In [15]:
import os
import requests
import ollama
import gradio as gr
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import anthropic

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
MODEL_CLAUDE = "claude-3-haiku-20240307"

In [26]:
technical_prompt = "Du bist ein Experte in der Softwareentwicklung für die Sprache Python, C# und C++. Du unterstützt die Softwareentwickler dabei ihren Code zu verbessern und bewertest ihren Programierstil."

In [27]:
user_prompt = "Analysiere und bewerte folgenden Code und erstelle einen Bericht in Markdown mit Verbesserungsvorschläge (ohne Code Beispiele). Bitte Stufe den Code ein ob er Beginner, Normal, Senior oder Expert Level hat:/n"
code = """using System.IdentityModel.Tokens.Jwt;
                using System.Security.Claims;
                using API.DTOs;
                using Blazored.LocalStorage;
                using Microsoft.AspNetCore.Components.Authorization;
                
                namespace Client.Components.Provider;
                
                public class CustomAuthenticationStateProvider : AuthenticationStateProvider
                {
                    private readonly ILocalStorageService _localStorage;
                    private readonly HttpClient _httpClient;
                
                    public CustomAuthenticationStateProvider(
                        ILocalStorageService localStorage,
                        HttpClient httpClient)
                    {
                        _localStorage = localStorage;
                        _httpClient = httpClient;
                    }
                
                    public override async Task<AuthenticationState> GetAuthenticationStateAsync()
                    {
                        var user = await _localStorage.GetItemAsync<UserDto>("user");
                
                        if (user == null || string.IsNullOrEmpty(user.Token))
                        {
                            return new AuthenticationState(new ClaimsPrincipal(new ClaimsIdentity()));
                        }
                
                        SetAuthorizationHeader(user.Token);
                
                        var claims = ParseClaimsFromJwt(user.Token);
                        var identity = new ClaimsIdentity(claims, "jwt");
                        var principal = new ClaimsPrincipal(identity);
                
                        return new AuthenticationState(principal);
                    }
                
                    public async Task SetAuthenticationState(UserDto user)
                    {
                        if (user != null)
                        {
                            SetAuthorizationHeader(user.Token);
                            var claims = ParseClaimsFromJwt(user.Token);
                            var identity = new ClaimsIdentity(claims, "jwt");
                            var principal = new ClaimsPrincipal(identity);
                            NotifyAuthenticationStateChanged(Task.FromResult(new AuthenticationState(principal)));
                        }
                        else
                        {
                            _httpClient.DefaultRequestHeaders.Authorization = null;
                            NotifyAuthenticationStateChanged(Task.FromResult(new AuthenticationState(new ClaimsPrincipal())));
                        }
                    }
                
                    private void SetAuthorizationHeader(string token)
                    {
                        _httpClient.DefaultRequestHeaders.Authorization = 
                            new System.Net.Http.Headers.AuthenticationHeaderValue("Bearer", token);
                    }
                
                    private IEnumerable<Claim> ParseClaimsFromJwt(string jwt)
                    {
                        var handler = new JwtSecurityTokenHandler();
                        var token = handler.ReadJwtToken(jwt);
                        return token.Claims;
                    }
                }"""

In [22]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyC0


In [54]:
def get_open_ai(model):
    if model=="llama":
        return OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
        
    return OpenAI()

In [58]:
def get_open_ai_model_version(model):
    if model=="llama":
        return MODEL_LLAMA
        
    return MODEL_GPT

In [57]:
def stream_open_ai(prompt, model):
    openai = get_open_ai(model)
    model_version = get_open_ai_model_version(model)
    
    messages = [
        {"role": "system", "content": technical_prompt},
        {"role": "user", "content": prompt}
    ]
    
    stream = openai.chat.completions.create(
        model=model_version,
        messages=messages,
        stream=True
    )
    
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [18]:
def stream_claude(prompt):
    claude = anthropic.Anthropic()
    result = claude.messages.stream(
        model=MODEL_CLAUDE,
        max_tokens=1000,
        temperature=0.7,
        system=technical_prompt,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [59]:
def analyze_code(code, model):
    prompt = user_prompt + code
    
    if model=="GPT" or model=="llama":
        result = stream_open_ai(prompt, model)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
        
    yield from result

In [60]:
view = gr.Interface(
    fn=analyze_code,
    inputs=[gr.Textbox(label="Your Code:"), gr.Dropdown(["GPT", "Claude", "llama"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7891
* To create a public link, set `share=True` in `launch()`.
